# Calculate distances between specific residues in pdb files

In [1]:
import pandas as pd
import numpy as np
from os.path import exists
from biopandas.pdb import PandasPdb

def sanitize_protein(protein_id):
    "Replace / in protein ID with _ for compatibility with file system"
    return protein_id.replace("/", "_")

def calculate_dist(pdb, XL):
    "Calculate the distance between residues in the PDB based on the crosslink"
    # Find the C atom for each residue
    res_1 = pdb.loc[pdb['residue_number'] == XL.Res1]
    res_2 = pdb.loc[pdb['residue_number'] == XL.Res2]
    
    res_1_coords = res_1[['x_coord', 'y_coord', 'z_coord']].to_numpy()
    res_2_coords = res_2[['x_coord', 'y_coord', 'z_coord']].to_numpy()
    
    return np.linalg.norm(res_1_coords - res_2_coords)

In [2]:
# Read in XL csv file to pandas DF
XLs = pd.read_csv("Union_XL1_XL2_PurifiedSECa-g_IEXcilia2inject2.3-newpeptidepos.csv", encoding= 'unicode_escape')

# Save all intramolecular crosslinks to a new DF
XLs_intra = XLs.loc[XLs['Protein1'] == XLs['Protein2']]

# Calculate the XL position in the protein primary sequence
XLs_intra['Res1'] = XLs_intra['PepPos1'] + XLs_intra['LinkPos1'] - 1
XLs_intra['Res2'] = XLs_intra['PepPos2'] + XLs_intra['LinkPos2'] - 1

# Removes all XL redundancies 
XLs_intra['newRes1'] = XLs_intra[["Res1", "Res2"]].max(axis=1)
XLs_intra['newRes2'] = XLs_intra[["Res1", "Res2"]].min(axis=1)
XLs_intra_nonred = XLs_intra.drop_duplicates(subset=['Protein1', 'Protein2', 'newRes1', 'newRes2'], keep="last")

C:\Users\Erin\AppData\Local\Temp/ipykernel_18744/2994176413.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  XLs_intra['Res1'] = XLs_intra['PepPos1'] + XLs_intra['LinkPos1'] - 1
C:\Users\Erin\AppData\Local\Temp/ipykernel_18744/2994176413.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  XLs_intra['Res2'] = XLs_intra['PepPos2'] + XLs_intra['LinkPos2'] - 1
C:\Users\Erin\AppData\Local\Temp/ipykernel_18744/2994176413.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [3]:
# Inspection of a specific protein 
XLs_intra_nonred.loc[XLs_intra_nonred['Protein1'] == 'PRKG1']

,Id,Score,Protein1,PepPos1,PepSeq1,LinkPos1,Protein2,PepPos2,PepSeq2,LinkPos2,Res1,Res2,newRes1,newRes2
7149,7150,153.24,PRKG1,211,KTIEDMVQK,1,PRKG1,637,QGKHPGR,3,211,639,639,211
8914,8915,137.84,PRKG1,469,KMANLEDK,1,PRKG1,477,KDYSHIK,1,469,477,477,469
8927,8928,137.80,PRKG1,786,LVINEIKHEQEKGK,12,PRKG1,492,KLGFGQFGSVYLVR,1,797,492,797,492
9230,9231,135.02,PRKG1,212,TIEDMVQKNYEANR,8,PRKG1,634,ILKQGK,3,219,636,636,219
9383,9384,133.90,PRKG1,226,TFMEQVNFFKSMNSEQK,10,PRKG1,637,QGKHPGR,3,235,639,639,235
10661,10662,121.68,PRKG1,435,TDGVISSISYKTFQK,11,PRKG1,365,ITNNCKIANLK,6,445,370,445,370
11540,11541,113.72,PRKG1,721,LIDQLLSKVPELR,8,PRKG1,325,DNLTKILGDK,5,728,329,728,329
11692,11693,112.13,PRKG1,236,SMNSEQKDSIASALINIK,7,PRKG1,346,WSFAKSDLLK,5,242,350,350,242
11739,11740,111.63,PRKG1,293,GDSFGEQALYVSSKR,14,PRKG1,275,EGTVSVWKDKK,8,306,282,306,282
12117,12118,107.19,PRKG1,506,NKDSGALYALK,2,PRKG1,477,KDYSHIK,1,507,477,507,477


In [4]:
# Takes the first 100 most crosslinked proteins
intra_XL_counts = XLs_intra_nonred['Protein1'].value_counts().reset_index().rename(columns = {'index': 'Protein1', 'Protein1': 'XL_count'})
intra_XL_counts[:100]

,Protein1,XL_count
0,DYH4,63
1,SPATA5,56
2,DYH3,50
3,DYH5,42
4,PRKG1,38
...,...,...
95,SPEF2,9
96,DPY30-1,9
97,Q24FH6,9
98,I7MCM4,9


In [5]:
def get_af_XL_counts(protein_XL_counts, threshold):
    protein = protein_XL_counts['Protein1']
    
    protein_XLs = XLs_intra_nonred.loc[XLs_intra_nonred['Protein1'] == protein]
    #print(protein_XLs)
    
    pdb_filepath = 'models/' + sanitize_protein(protein) + '-unrelaxed_model_1.pdb'
    if (not exists(pdb_filepath)):
        print(protein + ' PDB file not found. Skipping...')
        return None
    
    ppdb = PandasPdb().read_pdb(pdb_filepath) # reads pdb into pandas df
    pdbatom = ppdb.df['ATOM'] # extracts only the atom data type from pdb
    subpdb = pdbatom[(pdbatom['atom_name']=='CA')] # selects the only the CA atoms
    #print(subpdb)

    protein_XLs['af_distance'] = protein_XLs.apply(lambda x : calculate_dist(subpdb, x), axis=1)
    #print(protein_XLs)
    #print(protein_XLs[protein_XLs['af_distance'] < threshold])
    
    return protein_XLs[protein_XLs['af_distance'] < 30].count()['Id']

In [6]:
intra_XL_counts['af_XL_count'] = intra_XL_counts.apply(lambda x : get_af_XL_counts(x, 30), axis = 1)

DYH4 PDB file not found. Skipping...
SPATA5 PDB file not found. Skipping...
DYH3 PDB file not found. Skipping...
DYH5 PDB file not found. Skipping...
CFAP45 PDB file not found. Skipping...
DYH7 PDB file not found. Skipping...
EMAP5/6-1 PDB file not found. Skipping...
EMAP5/6-2 PDB file not found. Skipping...
EFhand-2 PDB file not found. Skipping...
TUBA PDB file not found. Skipping...
IFT139 PDB file not found. Skipping...
EFHC1/2-2 PDB file not found. Skipping...
DYH15 PDB file not found. Skipping...
DYH6 PDB file not found. Skipping...
HSP70 PDB file not found. Skipping...
SPAG17 PDB file not found. Skipping...
IFT122 PDB file not found. Skipping...
IFT144 PDB file not found. Skipping...
EPA1 PDB file not found. Skipping...
CFAP53 PDB file not found. Skipping...
IDP2  PDB file not found. Skipping...
UBXN2A/B PDB file not found. Skipping...
IFT81 PDB file not found. Skipping...
HYDIN PDB file not found. Skipping...
DYH30 PDB file not found. Skipping...
CCDC96 PDB file not found. Skipp

I7M3P1 PDB file not found. Skipping...
CAPN-3 PDB file not found. Skipping...
Q23EW5 PDB file not found. Skipping...
LRR4 PDB file not found. Skipping...
Q22LV8 PDB file not found. Skipping...
ARP3 PDB file not found. Skipping...
FMO1-6 PDB file not found. Skipping...
I7LWU2 PDB file not found. Skipping...
Q22KH2 PDB file not found. Skipping...
CAT1 PDB file not found. Skipping...
VMA2 PDB file not found. Skipping...
Q233E0 PDB file not found. Skipping...
NME1 PDB file not found. Skipping...
RPS7 PDB file not found. Skipping...
TTP80 PDB file not found. Skipping...
DYH23 PDB file not found. Skipping...
I7MIG3 PDB file not found. Skipping...
W7X817 PDB file not found. Skipping...
W7X6K3 PDB file not found. Skipping...
GST57 PDB file not found. Skipping...
NBP35  PDB file not found. Skipping...
I7M603 PDB file not found. Skipping...
CRN1 PDB file not found. Skipping...
CFAP52 PDB file not found. Skipping...
I7M0X0 PDB file not found. Skipping...
TCTEX1D1 PDB file not found. Skipping...
Q

MPK6 PDB file not found. Skipping...
DDI1 PDB file not found. Skipping...
MRNC05 PDB file not found. Skipping...
I7M7W9 PDB file not found. Skipping...
UMP1 PDB file not found. Skipping...
I7M9K3 PDB file not found. Skipping...
Q23TB4 PDB file not found. Skipping...
MRK1 PDB file not found. Skipping...
Q24I57 PDB file not found. Skipping...
Q22N79 PDB file not found. Skipping...
Q24E28 PDB file not found. Skipping...
Q22WZ2 PDB file not found. Skipping...
I7M2S1 PDB file not found. Skipping...
I7M893 PDB file not found. Skipping...
I7MAI3 PDB file not found. Skipping...
I7MKC2 PDB file not found. Skipping...
I7MHH0 PDB file not found. Skipping...
PYK2 PDB file not found. Skipping...
Q22N21 PDB file not found. Skipping...
I7MB37 PDB file not found. Skipping...
I7ME17 PDB file not found. Skipping...
I7MIJ4 PDB file not found. Skipping...
Q22W51 PDB file not found. Skipping...
I7MDI3 PDB file not found. Skipping...
Q24DL4 PDB file not found. Skipping...
MRNN09 PDB file not found. Skipping

In [7]:
intra_XL_counts[intra_XL_counts['af_XL_count'].notnull()]

,Protein1,XL_count,af_XL_count
4,PRKG1,38,36.0
5,EEF2,35,25.0
6,MRNC57,34,25.0
10,I7MJ59,30,15.0
11,BBC118,25,15.0
12,FTT18,25,21.0
13,EARS,24,24.0
16,Q23A15,20,12.0
19,KARS,18,12.0
21,PRKAR1A,18,18.0
